In [1]:
pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


RBF kernel

In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# RBF Kernel
def rbf_kernel(x, c, gamma=0.1):
    return np.exp(-gamma * np.linalg.norm(x - c)**2)

# Initial Projection
def initial_projection(X, centroids, gamma=0.1):
    M = len(centroids)
    projection = np.zeros((X.shape[0], M))
    for i, x in enumerate(X):
        for j, c in enumerate(centroids):
            projection[i, j] = rbf_kernel(x, c, gamma)
    return projection

WLS-SVC and Update Rules

In [3]:
class WLS_SVC:
    def __init__(self, C=1.0):
        self.C = C
    
    def fit(self, X, y):
        l, M = X.shape
        self.alpha = np.zeros(l)
        self.w = np.zeros(M)
        self.b = 0

        for _ in range(10):  # A fixed number of iterations for simplicity
            for i in range(l):
                e_i = y[i] - (X[i] @ self.w + self.b)
                a_i = 2 * self.alpha[i] * y[i] / (1 - y[i] * (X[i] @ self.w + self.b))
                self.w += a_i * y[i] * X[i]
                self.b += (y[i] - X[i] @ self.w)
                self.alpha[i] = min(max(self.alpha[i] + e_i, 0), self.C)
    
    def predict(self, X):
        return np.sign(X @ self.w + self.b)

# Helper function to find new centroids (support vectors)
def find_new_centroids(X, y, model):
    support_vectors = X[np.abs(y - model.predict(X)) > 0.1]  # Threshold for selecting new centroids
    return support_vectors


Iterative Scheme to Update the Classifier

In [4]:
def iterative_poker(X, y, initial_centroids, iterations=10, gamma=0.1):
    centroids = initial_centroids
    wls_svc = WLS_SVC()

    for _ in range(iterations):
        projection = initial_projection(X, centroids, gamma)
        wls_svc.fit(projection, y)
        new_centroids = find_new_centroids(X, y, wls_svc)
        centroids = np.vstack([centroids, new_centroids])
    
    return wls_svc

# Generate synthetic data
X, y = make_classification(n_samples=500, n_features=20, n_informative=15, n_redundant=5, random_state=42)
y = 2 * y - 1  # Convert labels to -1 and 1

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initial centroids
initial_centroids = X_train[np.random.choice(X_train.shape[0], 4, replace=False)]

# Apply POKER
poker_model = iterative_poker(X_train, y_train, initial_centroids)
poker_projection = initial_projection(X_test, initial_centroids)
poker_predictions = poker_model.predict(poker_projection)
poker_accuracy = accuracy_score(y_test, poker_predictions)

# Apply regular SVM
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)

print(f"POKER Accuracy: {poker_accuracy}")
print(f"SVM Accuracy: {svm_accuracy}")


C:\Users\KaraPardazesh\AppData\Local\Temp\ipykernel_3620\4081604373.py:14: RuntimeWarning: invalid value encountered in scalar divide
  a_i = 2 * self.alpha[i] * y[i] / (1 - y[i] * (X[i] @ self.w + self.b))


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 20)